In [1]:
import os

import alphashape
import gmsh
import h5py
import matplotlib.pyplot as plt
import meshio
import numpy as np
import shapely
import warnings

from concavehull import concavehull
from descartes import PolygonPatch
from dolfinx import cpp, fem, io, mesh, nls, plot
from mpi4py import MPI
from petsc4py import PETSc

import commons, geometry, grapher, utils
warnings.simplefilter('ignore')

In [2]:
markers = commons.SurfaceMarkers()
cell_types = commons.CellTypes()

### 2D Irregular Surface

In [3]:
def mesh_surface(coords, xmax=470, ymax=470):
    points = {}
    new_points = {}
    count = 0
    for row in coords:
        points[(row[0], row[1])] = count
        count += 1
    points_set = set(points.keys())
    no_full_neighbors = set()
    triangles = []
    for (x0, y0) in points_set:
        p0 = points[(x0, y0)]
        neighbors = [
            (int(x0 + 1), y0),
            (int(x0 + 1), int(y0 + 1)),
            (x0, int(y0 + 1))
        ]
        neighbor_points = [p0]
        for p in neighbors:
            v = points.get(p)
            neighbor_points.append(v)

        midpoint = (x0 + 0.5, y0 + 0.5)
        if midpoint[0] > xmax or midpoint[1] > ymax:
            continue
        points[midpoint] = count
        p2 = count
        count += 1
        for i in range(4):
            p0 = neighbor_points[i]
            if i == 3:
                p1 = neighbor_points[0]
            else:
                p1 = neighbor_points[i + 1]
            if not p0 is None and not p1 is None:
                triangles.append(
                    (p0, p1, p2)
                )
                

    return triangles, points

In [4]:
img = np.asarray(plt.imread('data/current_constriction/test11.tif')[:, :, 0], dtype=np.uint8)
img2 = img.copy()
img2[0:5, :] = 0
img2[-5:, :] = 0
img2[:, 0:5] = 0
img2[:, -5:,] = 0
coords = np.asarray(np.argwhere(img2 == 1), dtype=np.int32)

In [5]:
triangles, points = mesh_surface(coords)
cell_type = cell_types.triangle
cells = np.asarray(triangles, dtype=np.int32)
max_row_idx = max(points.values())
new_points = np.zeros((max_row_idx + 1, 2))

for k, v in points.items():
    new_points[v, :] = k
cell_data = np.zeros((len(triangles),), dtype=np.int32).tolist()
out_mesh = meshio.Mesh(points=new_points, cells={cell_type: cells}, cell_data={"name_to_read": [cell_data]})
out_mesh.write("trial.xdmf")

### 3D Surface Marking

### Irregular Surface Using gmsh

In [6]:
coords = np.asarray(np.argwhere(img2 == 1), dtype=np.int32)
xmax = 470
ymax = 470
all_points = {}
corner_points = [
    (0, 0, 0),
    (xmax, 0, 0),
    (xmax, ymax, 0),
    (0, ymax, 0)
]
other_points = []
count = 0
for row in coords:
    all_points[(row[0], row[1])] = count
    count += 1
gmsh_points = []
points_view = {int(v): (int(k[0]), int(k[1])) for k, v in all_points.items()}

In [7]:
graph = grapher.PixelGraph(points=points_view)
graph.build_graph()
graph.get_graph_pieces()
graph.n_pieces

120

#### Meshing with GMSH

In [8]:
Lx = 470
Ly = 470
Lz = 2
try:
    gmsh.finalize()
except:
    pass
gmsh.initialize()
gmsh.model.add('area')
z0_points = [
    (0, 0, 0),
    (Lx, 0, 0),
    (Lx, Ly, 0),
    (0, Ly, 0),
]
zL_points = [
    (0, 0, Lz),
    (Lx, 0, Lz),
    (Lx, Ly, Lz),
    (0, Ly, Lz),
]
points0 = []
points1 = []
lines = []

for i in range(4):
    points0.append(
        gmsh.model.occ.addPoint(*z0_points[i], meshSize=1)
    )
for i in range(4):
    points1.append(
        gmsh.model.occ.addPoint(*zL_points[i], meshSize=1)
    )
gmsh.model.occ.synchronize()
for i in range(-1, 3):
    lines.append(
        gmsh.model.occ.addLine(points0[i], points0[i + 1])
    )
for i in range(-1, 3):
    lines.append(
        gmsh.model.occ.addLine(points1[i], points1[i + 1])
    )
# 1 --> 5
lines.append(
    gmsh.model.occ.addLine(points0[1], points1[1])
)
# 2 --> 6
lines.append(
    gmsh.model.occ.addLine(points0[2], points1[2])
)
# 3 --> 7
lines.append(
    gmsh.model.occ.addLine(points0[3], points1[3])
)
# 0 --> 4
lines.append(
    gmsh.model.occ.addLine(points0[0], points1[0])
)
gmsh.model.occ.synchronize()

loops = []
# xy sides
loops.append(
    gmsh.model.occ.addCurveLoop(lines[:4])
)
loops.append(
    gmsh.model.occ.addCurveLoop(lines[4:8])
)
# xz sides
loops.append(
    gmsh.model.occ.addCurveLoop([lines[1]] + [lines[8]] + [lines[5]] + [lines[11]])
)
loops.append(
    gmsh.model.occ.addCurveLoop([lines[3]] + [lines[9]] + [lines[7]] + [lines[10]])
)
# yz sides
loops.append(
    gmsh.model.occ.addCurveLoop([lines[2]] + [lines[8]] + [lines[6]] + [lines[9]])
)
loops.append(
    gmsh.model.occ.addCurveLoop([lines[0]] + [lines[11]] + [lines[4]] + [lines[10]])
)
gmsh.model.occ.synchronize()

side_loops = []
insulated = []
right = []
left = []

for p in graph.pieces:
    if len(p) < 50:
        continue
    arr = []
    for c in p:
        arr.append(points_view[c])
    alpha_shape = alphashape.alphashape(arr, 0.25)
    exterior = alpha_shape.exterior
    hull = []
    for c in exterior.coords:
        hull.append((c[0], c[1]))
    hull_arr = np.asarray(hull)
    hull_points = []
    for pp in hull[:-1]:
        hull_points.append(
            gmsh.model.occ.addPoint(int(pp[0]), int(pp[1]), 0)
        )
    gmsh.model.occ.synchronize()
    hull_lines = []
    for i in range(-1, len(hull_points) - 1):
        hull_lines.append(
            gmsh.model.occ.addLine(hull_points[i], hull_points[i + 1])
        )
    gmsh.model.occ.synchronize()
    hull_loop = gmsh.model.occ.addCurveLoop(hull_lines)
    side_loops.append(hull_loop)
gmsh.model.occ.synchronize()

insulated = []
insulated += [gmsh.model.occ.addPlaneSurface((loops[0], *side_loops))]
left = [gmsh.model.occ.addPlaneSurface((vv, )) for vv in side_loops]
righ = [gmsh.model.occ.addPlaneSurface((loops[1], ))]
insulated += [gmsh.model.occ.addPlaneSurface((vv, )) for vv in loops[2:]]
gmsh.model.occ.synchronize()
lefttag = gmsh.model.addPhysicalGroup(2, left, markers.left_cc)
righttag = gmsh.model.addPhysicalGroup(2, right, markers.right_cc)
insulatedtag = gmsh.model.addPhysicalGroup(2, insulated, markers.insulated)
gmsh.model.occ.synchronize()
# extr = gmsh.model.occ.extrude(gmsh.model.getEntities(2), 0, 0, 25)
# gmsh.model.occ.synchronize()
# print(extr)
print(gmsh.model.occ.getEntities(2), gmsh.model.getEntities(3))
sloop = gmsh.model.occ.addSurfaceLoop(tuple([vv[1] for vv in gmsh.model.occ.getEntities(2)]))
volz = gmsh.model.occ.addVolume((sloop,))
gmsh.model.occ.synchronize()
vols = gmsh.model.occ.getEntities(3)
print(vols)
volz = [vv[1] for vv in vols]
gmsh.model.addPhysicalGroup(3, volz, 1)
gmsh.model.occ.synchronize()
gmsh.model.mesh.generate(3)
gmsh.write("trial.msh")
gmsh.finalize()

Error   : Gmsh has not been initialized


[(2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23), (2, 24), (2, 25), (2, 26)] []
[(3, 1)]
Info    : Meshing 1D...
Info    : [  0%] Meshing curve 1 (Line)
Info    : [ 10%] Meshing curve 2 (Line)
Info    : [ 10%] Meshing curve 3 (Line)
Info    : [ 10%] Meshing curve 4 (Line)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Line)
Info    : [ 10%] Meshing curve 7 (Line)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Line)
Info    : [ 10%] Meshing curve 10 (Line)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 10%] Meshing curve 12 (Line)
Info    : [ 10%] Meshing curve 13 (Line)
Info    : [ 10%] Meshing curve 14 (Line)
Info    : [ 10%] Meshing curve 15 (Line)
Info    : [ 10%] Meshing curve 16 (Line)
Info    : [ 10%] Meshing curve 17 (Line)
Info    : [ 10%] Meshing curve 18 (Line)
Info    :

Info    : [ 10%] Meshing surface 2 (Plane, Frontal-Delaunay)
Info    : [ 10%] Meshing surface 3 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 4 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 5 (Plane, Frontal-Delaunay)
Info    : [ 20%] Meshing surface 6 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 7 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 8 (Plane, Frontal-Delaunay)
Info    : [ 30%] Meshing surface 9 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 10 (Plane, Frontal-Delaunay)
Info    : [ 40%] Meshing surface 11 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 12 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 13 (Plane, Frontal-Delaunay)
Info    : [ 50%] Meshing surface 14 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 15 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 16 (Plane, Frontal-Delaunay)
Info    : [ 60%] Meshing surface 17 (Plane, Frontal-Delaunay)
Info    : [ 70%]

Info    : It. 34500 - 30629 nodes created - worst tet radius 1.08035 (nodes removed 0 3871)
Info    : It. 35000 - 31107 nodes created - worst tet radius 1.0798 (nodes removed 0 3893)
Info    : It. 35500 - 31593 nodes created - worst tet radius 1.07921 (nodes removed 0 3907)
Info    : It. 36000 - 32074 nodes created - worst tet radius 1.07852 (nodes removed 0 3926)
Info    : It. 36500 - 32539 nodes created - worst tet radius 1.07797 (nodes removed 0 3961)
Info    : It. 37000 - 33000 nodes created - worst tet radius 1.07741 (nodes removed 0 4000)
Info    : It. 37500 - 33476 nodes created - worst tet radius 1.07685 (nodes removed 0 4024)
Info    : It. 38000 - 33959 nodes created - worst tet radius 1.07621 (nodes removed 0 4041)
Info    : It. 38500 - 34432 nodes created - worst tet radius 1.07566 (nodes removed 0 4068)
Info    : It. 39000 - 34913 nodes created - worst tet radius 1.0794 (nodes removed 0 4087)
Info    : It. 39500 - 35392 nodes created - worst tet radius 1.10846 (nodes remove

Info    : It. 79500 - 73863 nodes created - worst tet radius 1.02168 (nodes removed 0 5637)
Info    : It. 80000 - 74349 nodes created - worst tet radius 1.02091 (nodes removed 0 5651)
Info    : It. 80500 - 74824 nodes created - worst tet radius 1.02018 (nodes removed 0 5676)
Info    : It. 81000 - 75303 nodes created - worst tet radius 1.0194 (nodes removed 0 5697)
Info    : It. 81500 - 75792 nodes created - worst tet radius 1.01861 (nodes removed 0 5708)
Info    : It. 82000 - 76274 nodes created - worst tet radius 1.08723 (nodes removed 0 5726)
Info    : It. 82500 - 76749 nodes created - worst tet radius 1.01694 (nodes removed 0 5751)
Info    : It. 83000 - 77240 nodes created - worst tet radius 1.01611 (nodes removed 0 5760)
Info    : It. 83500 - 77724 nodes created - worst tet radius 1.01522 (nodes removed 0 5776)
Info    : It. 84000 - 78215 nodes created - worst tet radius 1.02643 (nodes removed 0 5785)
Info    : It. 84500 - 78696 nodes created - worst tet radius 1.01344 (nodes remov

In [9]:
outdir = f'mesh/study_2/470-470-{Lz}_000-000-000/'
utils.make_dir_if_missing(outdir)
scale_factor = [0.045, 0.045, 0.05]
msh = meshio.read("trial.msh")

tria_mesh_unscaled = geometry.create_mesh(msh, cell_types.triangle)
tria_mesh_unscaled.write(f"{outdir}" + 'tria.xdmf')
# tria_mesh_scaled = geometry.scale_mesh(tria_mesh_unscaled, cell_types.triangle, scale_factor=scale_factor)
# tria_mesh_scaled.write(f"{outdir}" + 'tria.xdmf')

tetr_mesh_unscaled = geometry.create_mesh(msh, cell_types.tetra)
tetr_mesh_unscaled.write(f"{outdir}" + 'tetr.xdmf')
# tetr_mesh_scaled = geometry.scale_mesh(tetr_mesh_unscaled, cell_types.tetra, scale_factor=scale_factor)
# tetr_mesh_scaled.write(f"{outdir}" + 'tetr.xdmf')